# 🥋 Lekcja 39: TorchScript Tracing (Wyjście z Pythona)

Zwykły model PyTorch wymaga interpretera Pythona.
**TorchScript** kompiluje model do **Intermediate Representation (IR)** – formatu, który można uruchomić w C++ (biblioteka LibTorch) z niesamowitą wydajnością.

**Metoda Tracing (`torch.jit.trace`):**
1.  Dajesz modelowi przykładowe dane (`dummy_input`).
2.  PyTorch puszcza dane przez model i "nagrywa" wszystkie operacje, które zostały wykonane.
3.  Zapisuje to nagranie jako statyczny graf.

**Zaleta:** Działa z każdym kodem (nawet bibliotekami zewnętrznymi).
**Wada:** "Zabetonowuje" ścieżkę wykonania. Jeśli masz w kodzie `if x > 0`, a przykładowe dane były dodatnie, to w skompilowanym modelu `if` zniknie i zawsze wykona się wersja pozytywna!

Zasymulujemy ten "cichy błąd".

In [1]:
import torch
import torch.nn as nn

# 1. Definiujemy model z pułapką (Warunek IF)
class DynamicNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(10, 10)

    def forward(self, x):
        # Logika zależna od danych!
        if x.sum() > 0:
            return self.linear(x) * 2  # Ścieżka A
        else:
            return self.linear(x) - 100 # Ścieżka B

model = DynamicNet()
print("Model gotowy. Ma dwie różne ścieżki działania.")

Model gotowy. Ma dwie różne ścieżki działania.


## Wykonanie Tracingu (Nagrywanie)

Użyjemy danych **dodatnich** do nagrywania.
Oznacza to, że PyTorch zobaczy tylko **Ścieżkę A** (`* 2`).
Ścieżka B (`- 100`) zostanie zignorowana i usunięta z grafu, bo podczas nagrywania kod tam nie wszedł.

In [2]:
# Dane dodatnie (uruchomią if)
example_positive = torch.ones(1, 10)

# TRACING
# check_trace=False wyłącza sprawdzanie błędów (celowo, żeby pokazać problem)
traced_model = torch.jit.trace(model, example_positive, check_trace=False)

print("✅ Model skompilowany (Tracing).")
print(type(traced_model)) # RecursiveScriptModule

✅ Model skompilowany (Tracing).
<class 'torch.jit._trace.TopLevelTracedModule'>


C:\Users\takze\AppData\Local\Temp\ipykernel_28840\56579129.py:12: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


## Inspekcja Kodu (IR)

Możemy zajrzeć do środka skompilowanego modelu, używając `.code`.
Zobaczysz, że instrukcja `if` **zniknęła**. Została sama matematyka ze Ścieżki A.

In [3]:
print("--- KOD SKOMPILOWANEGO MODELU ---")
print(traced_model.code)

print("\nCzy widzisz tu instrukcję 'if'? NIE.")
print("Model zapamiętał tylko operacje: linear i mnożenie przez 2.")

--- KOD SKOMPILOWANEGO MODELU ---
def forward(self,
    x: Tensor) -> Tensor:
  linear = self.linear
  _0 = torch.mul((linear).forward(x, ), CONSTANTS.c0)
  return _0


Czy widzisz tu instrukcję 'if'? NIE.
Model zapamiętał tylko operacje: linear i mnożenie przez 2.


## Dowód Błędu (Silent Bug)

Teraz wrzucimy do modelu dane **ujemne**.
1.  Oryginalny model (Python) wejdzie w `else` i odejmie 100.
2.  Skompilowany model (JIT) **nie ma else**, więc wykona mnożenie przez 2 (błędnie).

To jest koszmar debugowania na produkcji.

In [4]:
# Dane ujemne
example_negative = -torch.ones(1, 10)

# 1. Oryginał (Poprawny)
out_python = model(example_negative)
print(f"Wynik Python (Poprawny): {out_python.mean().item():.2f}")
# Oczekujemy wartości ujemnej i przesuniętej o -100

# 2. Traced (Zepsuty)
out_jit = traced_model(example_negative)
print(f"Wynik JIT    (Błędny):   {out_jit.mean().item():.2f}")

if not torch.allclose(out_python, out_jit):
    print("\n🚨 KATASTROFA! Model skompilowany działa inaczej niż oryginał.")
    print("Tracing 'zamroził' logikę na podstawie danych przykładowych.")

Wynik Python (Poprawny): -99.92
Wynik JIT    (Błędny):   0.16

🚨 KATASTROFA! Model skompilowany działa inaczej niż oryginał.
Tracing 'zamroził' logikę na podstawie danych przykładowych.


## Zapisywanie i Ładowanie

Mimo tej wady, Tracing jest super, jeśli Twój model jest **statyczny** (np. zwykły ResNet czy BERT bez dziwnych if-ów).
Taki model zapisuje się do pliku, który nie wymaga kodu Pythona do działania.

In [5]:
# Zapisz do pliku
traced_model.save("traced_model.pt")
print("💾 Zapisano model do pliku 'traced_model.pt'.")

# Wczytaj (działa nawet jeśli usuniesz klasę DynamicNet z kodu!)
loaded_model = torch.jit.load("traced_model.pt")
print("📂 Wczytano model.")

# Działa tak samo
print(loaded_model(example_positive).mean())

💾 Zapisano model do pliku 'traced_model.pt'.
📂 Wczytano model.
tensor(-0.1769, grad_fn=<MeanBackward0>)


## 🥋 Black Belt Summary

1.  **`torch.jit.trace`**: Działa jak magnetofon. Nagrywa ścieżkę, którą przeszły dane przykładowe.
2.  **Zaleta:** Obsługuje każdą bibliotekę Pythonową (NumPy, Pandas) wewnątrz modelu, bo po prostu nagrywa wynik operacji jako stałą.
3.  **Wada:** Usuwa `if`, `while`, `for` (zależne od danych). Model staje się sztywny.
4.  **Kiedy używać?** W 95% przypadków (standardowe CNN, Transformery).
5.  **Co jeśli potrzebuję `if`?** Musisz użyć **`torch.jit.script`**, którego nauczymy się w następnej lekcji.